# Mini-Project II

### High Level

1. Build a SQLite database of restaurants, bars and various points of interest (POIs) in the ___ area
2. Discern which API (Foursquare or Yelp) has better coverage over the selected area.

### Low Level
1. Pull data about various POIs in the ___ area via API (Foursquare and Yelp).
2. Create SQLite database and store the data about the POIs
3. Explore the data using SQL or Pandas and discern which API has better coverage of the area.
4. Choose the top 10 POIs based on the popularity (number of reviews or average rating) (Yelp, Foursquare).

In [3]:
import numpy as np
import pandas as pd
import requests as re
import os
import sqlite3
import foursquare as fsq
import json
from IPython.display import JSON #JSON(response.json())

### API Setup

In [2]:
# set API keys
yelp_id = os.environ["YELP_ID"]
yelp_key = os.environ["YELP_KEY"]
fsq_id = os.environ["FOURSQUARE_CLIENT_ID"]
fsq_secret = os.environ["FOURSQUARE_CLIENT_SECRET"]

#### Test: Yelp API connection
https://github.com/mLewisLogic/foursquare

In [3]:
test_url = 'https://api.yelp.com/v3/autocomplete?text=del&latitude=37.786882&longitude=-122.399972'
test_yelp = re.get(test_url, headers={'Authorization':f'Bearer {yelp_key}'})

In [4]:
test_yelp

<Response [200]>

#### Test: Foursquare API client connection
https://github.com/mLewisLogic/foursquare

In [5]:
# Construct the client object
client = fsq.Foursquare(client_id=fsq_id, client_secret=fsq_secret)

In [6]:
# Test connection
client.venues.search(params={'query': 'coffee', 'll': '40.7233,-74.0030'}) # test search

{'venues': [{'id': '4a118d01f964a52023771fe3',
   'name': 'Kam Hing Coffee Shop 金興',
   'location': {'address': '118 Baxter St',
    'crossStreet': 'btwn Hester and Canal St',
    'lat': 40.71801668499095,
    'lng': -73.99920951068383,
    'labeledLatLngs': [{'label': 'display',
      'lat': 40.71801668499095,
      'lng': -73.99920951068383}],
    'distance': 669,
    'postalCode': '10013',
    'cc': 'US',
    'city': 'New York',
    'state': 'NY',
    'country': 'United States',
    'formattedAddress': ['118 Baxter St (btwn Hester and Canal St)',
     'New York, NY 10013',
     'United States']},
   'categories': [{'id': '4bf58dd8d48988d1e0931735',
     'name': 'Coffee Shop',
     'pluralName': 'Coffee Shops',
     'shortName': 'Coffee Shop',
     'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/coffeeshop_',
      'suffix': '.png'},
     'primary': True}],
   'delivery': {'id': '2461479',
    'url': 'https://www.seamless.com/menu/new-kam-hing-118-baxter-st-new-york/2

#### Find Overlapping Categories (between Yelp and FSQ)

In [7]:
# open yelp categories from file
with open('yelp_categories.json') as f:
    yelp_c = json.load(f)

y_categories = pd.json_normalize(data=yelp_c ,record_path=['parents']).drop_duplicates()

In [8]:
# rename Yelp category column
y_categories.rename({0:'name'}, axis='columns',inplace=True)#.str.contains('food')]

In [9]:
# pull FSQ categories
f_categories = client.venues.categories()

In [10]:
# create FSQ dataframe
f_categories = pd.json_normalize(data=f_categories, record_path=['categories','categories'])

In [11]:
# lowercase FSQ columns of interest
f_categories['name'] = f_categories['name'].str.lower()
f_categories['pluralName'] = f_categories['pluralName'].str.lower()

In [12]:
# find overlapping categories between Yelp and FSQ
ol_categories = f_categories[['id','name','pluralName']].merge(y_categories['name'], left_on='pluralName', right_on='name')

In [13]:
ol_categories.rename({'id':'fsq_id','name_x':'fsq_name','pluralName':'overlap','name_y':'yelp_name'}, axis='columns',inplace=True)

In [15]:
# display overlapping categories
ol_categories

,fsq_id,fsq_name,overlap,yelp_name
0,4bf58dd8d48988d181941735,museum,museums,museums
1,4bf58dd8d48988d17b941735,zoo,zoos,zoos
2,5267e4d9e4b0ec79466e48c7,festival,festivals,festivals
3,4bf58dd8d48988d1c4941735,restaurant,restaurants,restaurants
4,4bf58dd8d48988d116941735,bar,bars,bars
5,50327c8591d4c4b30a586d5d,brewery,breweries,breweries
6,4bf58dd8d48988d15b941735,farm,farms,farms
7,4bf58dd8d48988d163941735,park,parks,parks
8,4bf58dd8d48988d14b941735,winery,wineries,wineries
9,52f2ab2ebcbc57f1066b8b3f,lawyer,lawyers,lawyers


#### Test: data retrieval

In [16]:
# Foursquare
fsq_data = client.venues.search(params={'query':'bar', 
                                    'll':'43.653225,-79.383186',
                                    'radius':'250',
                                    'limit':'5',
                                    'categoryId':'4bf58dd8d48988d116941735'})
# categoryID: bar, food, shop & service: 4d4b7105d754a06378d81259, hotel: 4bf58dd8d48988d1fa931735

In [17]:
JSON(fsq_data)

<IPython.core.display.JSON object>

In [18]:
# Yelp
bus_search = 'https://api.yelp.com/v3/businesses/search'
yelp_data = re.get(bus_search,
                   headers={'Authorization':f'Bearer {yelp_key}'},
                   params={'term':'bar',
                          'latitude':'49.282730',
                          'longitude':'-123.120735',
                          'radius':'250',
                          'limit':'5'
                          })

In [19]:
JSON(yelp_data.json())

<IPython.core.display.JSON object>

#### Test: dataframe designs

In [29]:
#yelp normalization
yelp_table = pd.json_normalize(data=yelp_data.json(), record_path=['businesses','categories'],
                      meta=[['businesses','id'],
                            ['businesses','name'],
                            ['businesses','coordinates','latitude'],
                            ['businesses','coordinates','longitude'],
                            ['businesses','distance']])

mapping = {'alias':'y_category',
           'businesses.id':'yelp_bus_id',
           'businesses.name':'venue',
           'businesses.coordinates.latitude':'latitude',
           'businesses.coordinates.longitude':'longitude',
           'businesses.distance':'distance'}

yelp_table.rename(columns=mapping, inplace=True)
yelp_table.drop(columns='title', inplace=True)

In [30]:
yelp_table

,y_category,yelp_bus_id,venue,latitude,longitude,distance
0,lounges,mdt1tLbkZcOS2CsEbVF9Xg,Prohibition,49.28356,-123.11893,135.8965
1,cocktailbars,mdt1tLbkZcOS2CsEbVF9Xg,Prohibition,49.28356,-123.11893,135.8965
2,lounges,FgATf88_fEkJKLWR2aKmew,1927 Lobby Lounge,49.283419,-123.118791,160.459714
3,pubs,QrOGQtujSAcBRr9TM3ixiA,The Bottleneck,49.280545,-123.120718,242.971585
4,newcanadian,QrOGQtujSAcBRr9TM3ixiA,The Bottleneck,49.280545,-123.120718,242.971585
5,newcanadian,nV-MkqGnkH1fyxuVCs5Jpw,JOEY Burrard,49.282785,-123.123168,176.566958
6,lounges,nV-MkqGnkH1fyxuVCs5Jpw,JOEY Burrard,49.282785,-123.123168,176.566958
7,bars,8Wzyn8c4OZ4OANfKQNVGyw,Notch8 Restaurant & Bar,49.28372,-123.12111,128.684941


In [31]:
#fsq normalization
fsq_table = pd.json_normalize(data=fsq_data, record_path=['venues','categories'],
                     meta=[['venues','id'],
                           ['venues','name'],
                           ['venues','location','lat'],
                           ['venues','location','lng'],
                           ['venues','location','distance']])

fsq_table.drop(columns=['id','name','shortName','primary','icon.prefix','icon.suffix'], inplace=True)

mapping = {'pluralName':'f_category',
           'venues.id':'fsq_bus_id',
           'venues.name':'venue',
           'venues.location.lat':'latitude',
           'venues.location.lng':'longitude',
           'venues.location.distance':'distance'}

fsq_table.rename(columns=mapping, inplace=True)

In [32]:
fsq_table

,f_category,fsq_bus_id,venue,latitude,longitude,distance
0,Pubs,4af8da92f964a5204a1022e3,Blue Stone Grill & Bar,43.651187,-79.381139,280
1,Speakeasies,5828e2b7b244c83857222352,Bar Under the Stairs,43.654444,-79.38446,170
2,Speakeasies,5828e0255f78034bcf481af8,Bar Under the Stairs,43.654526,-79.384525,180
3,Bars,5266fe1411d224a9cb6cb7ae,Foundation Bar,43.655123,-79.383767,216
4,Shopping Malls,59bc5030f79faa5f1e233670,CF Bubbly Bar,43.654986,-79.381169,254


#### Define function for collecting data 

In [33]:
def get_data(city, query=None, category=None):
    '''sends get request to yelp & fsq, returns json files'''
    
    # lat & longitudes for city centres: vancouver, toronto, new york city, and new westminster
    c = {'vancouver':('49.282730','-123.120735'),
    'toronto':('43.653225','-79.383186'),
    'nyc':('40.712776','-74.005974'),
    'new westminster':('49.205719','-122.910957')}
    
    try:     
        latitude, longitude = c[city.lower()]
            
        # YELP request parameters
        p={'latitude':latitude,
           'longitude':longitude,
           'radius':'1000'} # 1 km radius
        
        # add category to GET request if in function argument (convert to correct format)
        if (category != None):
            p['categories'] = ol_categories[ol_categories['overlap']==category]['yelp_name'].iat[0]
        
        # add query to GET request if in function arguement (already formatted correctly)
        if (query != None):
            p['term'] = query

        # YELP GET request / search
        data = re.get('https://api.yelp.com/v3/businesses/search',
                     headers={'Authorization':f'Bearer {yelp_key}'},
                     params=p)
        
        # format response into JSON
        data_yelp = data.json()
        print(p)

        # FSQ request parameters
        p = {'ll':f'{latitude},{longitude}',
            'radius':'1000'} # 1 km radius
        
        # add category to GET request if in function argument (convert to correct format)
        if (category != None):
            p['categoryId'] = ol_categories[ol_categories['overlap']==category]['fsq_id'].iat[0]

        # add query to GET request if in function arguement (convert to correct format)
        if (query != None):
            p['query'] = ol_categories[ol_categories['overlap']==query]['fsq_name'].iat[0]

        # FSQ GET request / search
        data_fsq = client.venues.search(params=p)
        print(p)

    except:
        print('some error occurred')
    
    # return fields appropriately based on input arguements, all indicated variables MUST be return for next function to work.
    finally:
        if (query != None) & (category != None):
            return data_yelp, data_fsq, city.lower(), query.lower(), category.lower()
        elif (query != None):
            category = ''
        else:
            query = ''
        
        return data_yelp, data_fsq, city.lower(), query.lower(), category.lower()

In [45]:
# test the get_data() function
test = get_data(category='bars',city='torOnto')

{'latitude': '43.653225', 'longitude': '-79.383186', 'radius': '1000', 'categories': 'bars'}
{'ll': '43.653225,-79.383186', 'radius': '1000', 'categoryId': '4bf58dd8d48988d116941735'}


In [46]:
len(test) # 5 = data_yelp, data_fsq, city, query, category

5

In [47]:
JSON(test[0]) # examine output

<IPython.core.display.JSON object>

In [49]:
JSON(test[1]) # examine output

<IPython.core.display.JSON object>

#### Define function for normalizing data

In [118]:
def normalize_data(data):

    # unpack input from get_data() function
    data_yelp, data_fsq, city, query, category = data
        
    # create YELP dataframe from JSON
    table_yelp = pd.json_normalize(data=data_yelp, record_path=['businesses','categories'],
                  meta=[['businesses','id'],
                        ['businesses','name'],
                        ['businesses','coordinates','latitude'],
                        ['businesses','coordinates','longitude'],
                        ['businesses','distance']])
    
    # drop uneeded columns
    table_yelp.drop(columns='title', inplace=True, errors='ignore')

    # mapping schema for renaming columns
    mapping = {'alias':'y_cat',
               'businesses.id':'yelp_bus_id',
               'businesses.name':'venue',
               'businesses.coordinates.latitude':'latitude',
               'businesses.coordinates.longitude':'longitude',
               'businesses.distance':'distance'}
    
    # rename columns
    table_yelp.rename(columns=mapping, inplace=True, errors='ignore')
    
    # json_normalize created duplicate entries, due to how the 'categories' node is unpacked from the JSON file
    table_yelp.drop_duplicates(['yelp_bus_id','latitude','longitude'], inplace=True)

    # create FSQ dataframe
    table_fsq = pd.json_normalize(data=data_fsq, record_path=['venues','categories'],
                 meta=[['venues','id'],
                       ['venues','name'],
                       ['venues','location','lat'],
                       ['venues','location','lng'],
                       ['venues','location','distance']])
    
    # drop uneeded columns
    table_fsq.drop(columns=['id','name','shortName','primary','icon.prefix','icon.suffix'], inplace=True, errors='ignore')
        
    # mapping schema for renaming columns
    mapping = {'pluralName':'f_cat',
               'venues.id':'fsq_bus_id',
               'venues.name':'venue',
               'venues.location.lat':'latitude',
               'venues.location.lng':'longitude',
               'venues.location.distance':'distance'}
    
    # rename the columns
    table_fsq.rename(columns=mapping, inplace=True, errors='ignore')
    
    # add accessory columns based on search arguements for get_data() function
    table_yelp['city'] = table_fsq['city'] = city
    table_yelp['source'] = 'yelp'
    table_fsq['source'] = 'fsq'
    table_yelp['query'] = table_fsq['query'] = query
    table_yelp['category'] = table_fsq['category'] = category
    
    # fill empty cells with NaN
    table_yelp.replace('', np.nan, inplace = True)
    table_fsq.replace('', np.nan, inplace=True)
    
    return table_yelp, table_fsq

In [40]:
# test normalize_data() function
test = normalize_data(test)

In [41]:
len(test) # 2 = table_yelp, table_fsq

2

In [42]:
# check output for table_yelp
test[0].head()

,y_cat,yelp_bus_id,venue,latitude,longitude,distance,city,source,query,category
0,japanese,RtUvSWO_UZ8V3Wpj0n077w,KINKA IZAKAYA ORIGINAL,43.660450,-79.379150,862.663552,toronto,yelp,NaN,bars
2,breakfast_brunch,cQK9M2JAwETQnnBoYyua5A,The Senator,43.655740,-79.378960,438.872684,toronto,yelp,NaN,bars
5,cocktailbars,yY3jNsrpCyKTqQuRuLV8gw,Cactus Club Cafe,43.649504,-79.382518,417.208551,toronto,yelp,NaN,bars
7,bistros,O-uIEuv7JLUHajkemx_sVw,Beerbistro,43.649450,-79.377159,641.351219,toronto,yelp,NaN,bars
9,newcanadian,Sk0Ycurq7D083Qs8x65b_Q,George Restaurant,43.653230,-79.374460,728.604682,toronto,yelp,NaN,bars


In [43]:
# check output for table_fsq
test[1].head()

,f_cat,fsq_bus_id,venue,latitude,longitude,distance,city,source,query,category
0,Italian Restaurants,4d306dd82748b60c62b6dba0,Trattoria Mercatto,43.654453,-79.380974,224,toronto,fsq,NaN,bars
1,Restaurants,4ad4c05df964a52059f620e3,Canoe,43.647452,-79.381320,659,toronto,fsq,NaN,bars
2,Italian Restaurants,4bbe0c134e069c745a829fe3,Metropolitan Resto Bar,43.650062,-79.377181,598,toronto,fsq,NaN,bars
3,Cocktail Bars,4ad4c05df964a5203df620e3,Piper's Gastropub,43.645468,-79.381779,870,toronto,fsq,NaN,bars
4,Speakeasies,50322b6ae4b09116a296568c,Soho House Toronto,43.648734,-79.386541,568,toronto,fsq,NaN,bars


#### Design Search Criteria for API calls (all possible permutations)

In [79]:
# Reminder what categories overlap between APIs
ol_categories['overlap']

0         museums
1            zoos
2       festivals
3     restaurants
4            bars
5       breweries
6           farms
7           parks
8        wineries
9         lawyers
10       airports
11         hotels
Name: overlap, dtype: object

In [80]:
# Select a subset of all overlapping categories to search for: 
combo = [ol_categories['overlap'][i] for i in [0,1,3,4,5,9,10,11]]
combo

['museums',
 'zoos',
 'restaurants',
 'bars',
 'breweries',
 'lawyers',
 'airports',
 'hotels']

In [81]:
# each search will take input = city, query (optional), category (optional)
queries = {'city':['vancouver','toronto','nyc','new westminster'],
           'query': combo,
           'category': combo}

In [82]:
# design iterator to create all possible permutations of our: city, query, category 
import itertools as it

allNames = sorted(queries)
combinations = it.product(*(queries[Name] for Name in allNames))
combo = list(combinations)

In [83]:
# query, city, category
combo[:10]

[('museums', 'vancouver', 'museums'),
 ('museums', 'vancouver', 'zoos'),
 ('museums', 'vancouver', 'restaurants'),
 ('museums', 'vancouver', 'bars'),
 ('museums', 'vancouver', 'breweries'),
 ('museums', 'vancouver', 'lawyers'),
 ('museums', 'vancouver', 'airports'),
 ('museums', 'vancouver', 'hotels'),
 ('museums', 'toronto', 'museums'),
 ('museums', 'toronto', 'zoos')]

#### Edit combos to suite needs

In [84]:
# turn tuples into lists
combo = [list(i) for i in combo]

In [85]:
# remove last element in each nested list if it matches the first element
for i in combo:
    if i[0] != i[2]:
        del i[2]

In [86]:
# if nested list is less than three elements in length, append 'None' to end
for i in combo:
    if len(i)<3:
        i.append(None)

In [87]:
# if last element in a nested list is 'None' reverse the list
for i in combo:
    if i[2] == None:
        combo.append(i[::-1])

In [88]:
# swap the first and second elements in each list
for i in combo:
    i[0], i[1] = i[1], i[0]

In [89]:
# sort list alphabetically by city
combo = sorted(combo, key=lambda x: x[0])

In [91]:
# drop duplicate lists
res = []
[res.append(x) for x in combo if x not in res]

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

In [96]:
len(res)

96

In [103]:
# now we have all our desired combos for searching the APIs
res[:24] # 24 searches per city: [city, query, category]

[['new westminster', 'museums', 'museums'],
 ['new westminster', 'museums', None],
 ['new westminster', 'zoos', None],
 ['new westminster', 'zoos', 'zoos'],
 ['new westminster', 'restaurants', None],
 ['new westminster', 'restaurants', 'restaurants'],
 ['new westminster', 'bars', None],
 ['new westminster', 'bars', 'bars'],
 ['new westminster', 'breweries', None],
 ['new westminster', 'breweries', 'breweries'],
 ['new westminster', 'lawyers', None],
 ['new westminster', 'lawyers', 'lawyers'],
 ['new westminster', 'airports', None],
 ['new westminster', 'airports', 'airports'],
 ['new westminster', 'hotels', None],
 ['new westminster', 'hotels', 'hotels'],
 ['new westminster', None, 'museums'],
 ['new westminster', None, 'zoos'],
 ['new westminster', None, 'restaurants'],
 ['new westminster', None, 'bars'],
 ['new westminster', None, 'breweries'],
 ['new westminster', None, 'lawyers'],
 ['new westminster', None, 'airports'],
 ['new westminster', None, 'hotels']]

#### Construct 1 YELP dataframe and 1 FSQ dataframe, per input (above)

In [119]:
# set empty lists for storing all created dataframes (from function runs)
yelp_jsons = []
fsq_jsons = []
yelp_dfs = []
fsq_dfs = []

In [120]:
counter = 1

# for each search combo
for i in range(len(res)):
        
    # unpack combo into arguements
    city, query, category = res[i]
    
    # visually enumarate which search (of 96) we are on, followed by the arguements used
    print(counter, city, query, category)
    
    # run API calls and generate JSONs
    out = get_data(city, query, category) # out = yelp_json, fsq_json, city, query, category
    yelp_j = out[0], out[2], out[3], out[4] 
    fsq_j = out[1], out[2], out[3], out[4] 
    
    # save JSONs to their lists
    yelp_jsons.append(yelp_j)
    fsq_jsons.append(fsq_j)
    
    # create dataframes from json files
    out = normalize_data(out)
    
    # save dataframes to their lists
    yelp_dfs.append(out[0])
    fsq_dfs.append(out[1])
    
    # increment counter + 1, repeat until all combos have been searched
    counter+=1
    print('\n')

1 new westminster museums museums
{'latitude': '49.205719', 'longitude': '-122.910957', 'radius': '1000', 'categories': 'museums', 'term': 'museums'}
{'ll': '49.205719,-122.910957', 'radius': '1000', 'categoryId': '4bf58dd8d48988d181941735', 'query': 'museum'}


2 new westminster museums None
{'latitude': '49.205719', 'longitude': '-122.910957', 'radius': '1000', 'term': 'museums'}
{'ll': '49.205719,-122.910957', 'radius': '1000', 'query': 'museum'}


3 new westminster zoos None
{'latitude': '49.205719', 'longitude': '-122.910957', 'radius': '1000', 'term': 'zoos'}
{'ll': '49.205719,-122.910957', 'radius': '1000', 'query': 'zoo'}


4 new westminster zoos zoos
{'latitude': '49.205719', 'longitude': '-122.910957', 'radius': '1000', 'categories': 'zoos', 'term': 'zoos'}
{'ll': '49.205719,-122.910957', 'radius': '1000', 'categoryId': '4bf58dd8d48988d17b941735', 'query': 'zoo'}


5 new westminster restaurants None
{'latitude': '49.205719', 'longitude': '-122.910957', 'radius': '1000', 'term

In [122]:
len(res) == len(yelp_jsons) == len(fsq_jsons) == len(yelp_dfs) == len(fsq_dfs) # all run & saved!

True

In [123]:
# Save our valuable JSONs to Files:
with open('yelp_jsons.txt', 'w') as outfile: ## write jsons to file
    json.dump(yelp_jsons, outfile)
with open('fsq_jsons.txt', 'w') as outfile: ## write jsons to file
    json.dump(fsq_jsons, outfile)

In [124]:
# merge created dataframes
yelp_master = pd.concat(yelp_dfs)
fsq_master = pd.concat(fsq_dfs)

In [126]:
# how long is each dataframe?
print(len(yelp_master), len(fsq_master))

1173 1216


In [133]:
yelp_master.head()

,yelp_bus_id,venue,latitude,longitude,distance,city,source,query,category
0,LD40feOYdj_dOcYN7hRhhw,Anvil Centre,49.201501,-122.911348,462.557651,new westminster,yelp,museums,museums
3,J7o-oykZz6bbq7jmfm4wzQ,Irving House,49.207882,-122.906235,418.95253,new westminster,yelp,museums,museums
5,ABw5g-La8fRWPV-7JSpyTw,The Fraser River Discovery Centre,49.200296,-122.911552,584.607287,new westminster,yelp,museums,museums
7,ByfeSlhDDEngYlEXcR-71A,New Westminster Museum and Archives,49.201501,-122.911348,462.557651,new westminster,yelp,museums,museums
8,zJU7VoaU8i1xpPw_Qmxaqw,New Westminster Museum and Archives,49.207922,-122.906401,411.77859,new westminster,yelp,museums,museums


In [136]:
fsq_master.head()

,fsq_bus_id,venue,latitude,longitude,distance,city,source,query,category
0,4e879627f9f4b60d1a6dcef4,"Zoomies, Dog Daycare and Training Centre",49.202584,-122.913966,411,new westminster,fsq,zoos,NaN
1,51435bade4b03a1cf03b53cd,Zoom Hair Studio,49.203674,-122.908447,291,new westminster,fsq,zoos,NaN
0,4baffc7ff964a52041303ce3,Stefano's Restaurant,49.205335,-122.90419,494,new westminster,fsq,restaurants,NaN
1,4b5545ebf964a5207ee027e3,The BoatHouse Restaurant,49.199736,-122.912409,674,new westminster,fsq,restaurants,NaN
2,4c65fbd88e9120a18bfbd664,The Old Bavaria Haus Restaurant,49.208352,-122.914272,379,new westminster,fsq,restaurants,NaN


In [130]:
len(yelp_master.columns) == len(fsq_master.columns) # arranged a bit different but same number of cols

True

In [132]:
# drop cols we won't be using for analysis
yelp_master.drop(columns='y_cat', inplace=True)
fsq_master.drop(columns='f_cat', inplace=True)

KeyError: "['y_cat'] not found in axis"

In [135]:
# rearrange columns to align between dfs
fsq_master = fsq_master[['fsq_bus_id','venue','latitude','longitude','distance','city','source','query','category']]

In [137]:
# save dataframes to csv
yelp_master.to_csv('./yelp_master.csv')
fsq_master.to_csv('./fsq_master.csv')

#### YELP: Add rating and review count

In [138]:
# list of all unique business ids
id_yelp = yelp_master['yelp_bus_id'].drop_duplicates().astype(str).tolist()

In [139]:
len(id_yelp)

607

In [140]:
# retrieve detailed JSONS with business ratings and reviews 
yelp_details = [re.get(f'https://api.yelp.com/v3/businesses/{ID}',
             headers={'Authorization':f'Bearer {yelp_key}'}).json() for ID in id_yelp]

In [141]:
# save detailed JSONs to file
with open('yelp_json_details.txt', 'w') as outfile:
    json.dump(yelp_details, outfile)

In [142]:
# view a JSON file
JSON(yelp_details[0])

<IPython.core.display.JSON object>

In [143]:
# Normalize all JSONs into dataframes
dataframes = [pd.json_normalize(data=yelp_details[i]) for i in range(len(yelp_details))]

In [144]:
# create master dataframe of detailed JSON data
complete_df = pd.concat(dataframes)

In [146]:
# merge desired columns from detailed dataframe to master
yelp_master_A = pd.merge(yelp_master, complete_df[['id','review_count','rating']], how='left', left_on='yelp_bus_id', right_on='id').drop(columns='id')

In [147]:
#check dataframe
yelp_master_A.head()

,yelp_bus_id,venue,latitude,longitude,distance,city,source,query,category,review_count,rating
0,LD40feOYdj_dOcYN7hRhhw,Anvil Centre,49.201501,-122.911348,462.557651,new westminster,yelp,museums,museums,4,4.5
1,J7o-oykZz6bbq7jmfm4wzQ,Irving House,49.207882,-122.906235,418.95253,new westminster,yelp,museums,museums,4,5.0
2,ABw5g-La8fRWPV-7JSpyTw,The Fraser River Discovery Centre,49.200296,-122.911552,584.607287,new westminster,yelp,museums,museums,1,5.0
3,ByfeSlhDDEngYlEXcR-71A,New Westminster Museum and Archives,49.201501,-122.911348,462.557651,new westminster,yelp,museums,museums,1,3.0
4,zJU7VoaU8i1xpPw_Qmxaqw,New Westminster Museum and Archives,49.207922,-122.906401,411.77859,new westminster,yelp,museums,museums,1,3.0


In [149]:
len(yelp_master_A) == len(yelp_master) # check to make sure no data was lost

True

In [162]:
# check for missing values
print(yelp_master_A['review_count'].isnull().values.any())
print(yelp_master_A['rating'].isnull().values.any())

False
False


In [165]:
# check contents
print(yelp_master_A['review_count'].sort_values())
print(yelp_master_A['rating'].sort_values())

662       1
134       1
135       1
445       1
633       1
       ... 
563    2904
583    2904
777    2904
463    6136
449    6268
Name: review_count, Length: 1173, dtype: int64
345     1.0
197     1.0
374     1.0
717     1.0
448     1.0
       ... 
167     5.0
168     5.0
181     5.0
992     5.0
1012    5.0
Name: rating, Length: 1173, dtype: float64


In [166]:
# save resulting master dataframe to file
yelp_master_A.to_csv('./yelp_master_A.csv')

#### FSQ: Adding review count and ratings from FSQ is more challenging, so first I will store our current dataframes in a SQLite database.

## Setup SQLite Database

In [1]:
import sqlite3
from sqlite3 import Error

def create_connection(path):
    connection = None
    try:
        connection = sqlite3.connect(path)
        print("Connection to SQLite DB successful")
    except Error as e:
        print(f"The error '{e}' occurred")

    return connection

In [2]:
connection = create_connection("./sm_app.sqlite")

Connection to SQLite DB successful


In [169]:
def execute_query(connection, query):
    cursor = connection.cursor()
    try:
        cursor.execute(query)
        connection.commit()
        print("Query executed successfully")
    except Error as e:
        print(f"The error '{e}' occurred")

In [170]:
def execute_read_query(connection, query):
    cursor = connection.cursor()
    result = None
    try:
        cursor.execute(query)
        result = cursor.fetchall()
        return result
    except Error as e:
        print(f"The error '{e}' occurred")

In [171]:
# save dataframes to database
yelp_master_A.to_sql('yelp', connection, if_exists='replace')

In [172]:
fsq_master.to_sql('fsq', connection, if_exists='replace')

In [179]:
connection.close()

In [633]:
# create DB
from sqlalchemy import create_engine

# FSQ: Add Rating and Review Count
A free Foursquare account can only make 50 requests a day to the premium endpoint: venues/VENUE_ID

Therefore, I created 3 free FSQ accounts, and a function to send 50 get requests per account per day. Therefore I'll collect 150 JSON files per day (that's 1 JSON per unique business ID), until all 858 unique business IDs contained in our fsq dataframe have been accounted for.

This will take 858/150 ~= 6 days.

**See script created for this purpose called 'fsq_collect.py'**

Once script done collecting data, the following is processed...

In [205]:
# establish sql connection
connection = create_connection("./sm_app.sqlite")

Connection to SQLite DB successful


In [4]:
# read in fsq dataframe
fsq_master = pd.read_sql("SELECT * from fsq", connection, index_col='index')

In [5]:
# read in fsq details
with open('fsq_details860.txt') as f:
    fsq_jd = json.load(f)

In [6]:
len(fsq_jd)

860

In [7]:
# check read
len(fsq_master)

1216

In [8]:
fsq_master.head()

,fsq_bus_id,venue,latitude,longitude,distance,city,source,query,category
index,,,,,,,,,
0,4e879627f9f4b60d1a6dcef4,"Zoomies, Dog Daycare and Training Centre",49.202584,-122.913966,411,new westminster,fsq,zoos,None
1,51435bade4b03a1cf03b53cd,Zoom Hair Studio,49.203674,-122.908447,291,new westminster,fsq,zoos,None
0,4baffc7ff964a52041303ce3,Stefano's Restaurant,49.205335,-122.904190,494,new westminster,fsq,restaurants,None
1,4b5545ebf964a5207ee027e3,The BoatHouse Restaurant,49.199736,-122.912409,674,new westminster,fsq,restaurants,None
2,4c65fbd88e9120a18bfbd664,The Old Bavaria Haus Restaurant,49.208352,-122.914272,379,new westminster,fsq,restaurants,None


In [9]:
# view a JSON file
JSON(fsq_jd[0])

<IPython.core.display.JSON object>

In [10]:
# Normalize all JSONs into dataframes
dataframes = [pd.json_normalize(data=fsq_jd[i]) for i in range(len(fsq_jd))]

In [11]:
# create master dataframe of detailed JSON data
complete_df = pd.concat(dataframes)

In [12]:
# extract needed data from detailed DF
df_segment = complete_df[['venue.id','venue.ratingSignals','venue.rating']].rename(columns={'venue.id':'fsq_bus_id','venue.ratingSignals':'review_count', 'venue.rating':'rating'})

In [14]:
# merge desired columns from detailed dataframe to master
fsq_master_A = pd.merge(fsq_master, df_segment, how='left', on='fsq_bus_id')

In [16]:
#check dataframe
fsq_master_A.head()

,fsq_bus_id,venue,latitude,longitude,distance,city,source,query,category,review_count,rating
0,4e879627f9f4b60d1a6dcef4,"Zoomies, Dog Daycare and Training Centre",49.202584,-122.913966,411,new westminster,fsq,zoos,None,NaN,NaN
1,51435bade4b03a1cf03b53cd,Zoom Hair Studio,49.203674,-122.908447,291,new westminster,fsq,zoos,None,NaN,NaN
2,4baffc7ff964a52041303ce3,Stefano's Restaurant,49.205335,-122.904190,494,new westminster,fsq,restaurants,None,18.0,6.1
3,4b5545ebf964a5207ee027e3,The BoatHouse Restaurant,49.199736,-122.912409,674,new westminster,fsq,restaurants,None,51.0,6.5
4,4c65fbd88e9120a18bfbd664,The Old Bavaria Haus Restaurant,49.208352,-122.914272,379,new westminster,fsq,restaurants,None,25.0,7.9


In [17]:
len(fsq_master) == len(fsq_master_A) # check to make sure no data was lost

True

In [18]:
# check for missing values
print(fsq_master_A['review_count'].isnull().values.any())
print(fsq_master_A['rating'].isnull().values.any())

True
True


In [19]:
# check contents
print(fsq_master_A['review_count'].sort_values())
print(fsq_master_A['rating'].sort_values())

792     0.0
14      0.0
381     0.0
787     2.0
213     3.0
       ... 
1193    NaN
1196    NaN
1197    NaN
1198    NaN
1214    NaN
Name: review_count, Length: 1216, dtype: float64
618     4.6
786     4.9
1106    4.9
803     4.9
919     4.9
       ... 
1193    NaN
1196    NaN
1197    NaN
1198    NaN
1214    NaN
Name: rating, Length: 1216, dtype: float64


In [22]:
fsq_master_A.rating.isnull().value_counts()

False    691
True     525
Name: rating, dtype: int64

In [23]:
# save resulting dataframe to file
fsq_master_A.to_csv('./fsq_master_A.csv')

In [24]:
# export to database as well
fsq_master_A.to_sql('fsq', connection, if_exists='replace')

# Data Analysis

In [27]:
fdb = pd.read_sql_query("SELECT * from fsq", connection, index_col='index')
ydb = pd.read_sql_query("SELECT * from yelp", connection, index_col='index')

In [28]:
fdb.head()

,fsq_bus_id,venue,latitude,longitude,distance,city,source,query,category,review_count,rating
index,,,,,,,,,,,
0,4e879627f9f4b60d1a6dcef4,"Zoomies, Dog Daycare and Training Centre",49.202584,-122.913966,411,new westminster,fsq,zoos,None,NaN,NaN
1,51435bade4b03a1cf03b53cd,Zoom Hair Studio,49.203674,-122.908447,291,new westminster,fsq,zoos,None,NaN,NaN
2,4baffc7ff964a52041303ce3,Stefano's Restaurant,49.205335,-122.904190,494,new westminster,fsq,restaurants,None,18.0,6.1
3,4b5545ebf964a5207ee027e3,The BoatHouse Restaurant,49.199736,-122.912409,674,new westminster,fsq,restaurants,None,51.0,6.5
4,4c65fbd88e9120a18bfbd664,The Old Bavaria Haus Restaurant,49.208352,-122.914272,379,new westminster,fsq,restaurants,None,25.0,7.9


In [25]:
ydb.head()

NameError: name 'ydb' is not defined

## Search by query, with category (count)

#### FSQ

In [103]:
# vancouver


In [104]:
# toronto

In [105]:
# new westminster

In [106]:
# nyc

In [107]:
# overall (regardless of city)

#### YELP

In [103]:
# vancouver

In [104]:
# toronto

In [105]:
# new westminster

In [106]:
# nyc

In [107]:
# overall (regardless of city)

## Search by query, without category (count)

#### FSQ

In [103]:
# vancouver

In [104]:
# toronto

In [105]:
# new westminster

In [106]:
# nyc

In [107]:
# overall (regardless of city)

#### YELP

In [103]:
# vancouver

In [104]:
# toronto

In [105]:
# new westminster

In [106]:
# nyc

In [107]:
# overall (regardless of city)

## Search by category, without query (count)

#### FSQ

In [103]:
# vancouver


In [104]:
# toronto

In [105]:
# new westminster

In [106]:
# nyc

In [107]:
# overall (regardless of city)

#### YELP

In [103]:
# vancouver

In [104]:
# toronto

In [105]:
# new westminster

In [106]:
# nyc

In [107]:
# overall (regardless of city)

## Search by category, without query (count)

## Top 10 POIs based on popularity

## Compare APIs, Summary table
result counts
1. query alone        - cities - total  ----avg distance
2. category alone
3. query and category
4. overall

avg distance to location
1. fsq v yelp



## Challenges
1. API limits
* fsq limit, sleeper function, not realizing the venue (by id) endpoint is premium
* incredibly paranoid i wouldn't get results. Rarely ran code
* would go with yelp - details endpoint not premium
2. Not reading instructions clearly
* Did not see 'missed the top 10 POIs based on popularity' part
3. Understanding JSON file structure
4. 

In [629]:
engine = create_engine('sqlite:./sm_app.sqlite', echo=True)

ArgumentError: Could not parse rfc1738 URL from string './sm_app.sqlite'

In [ ]:
create_engine()

In [501]:
y[1]

,f_cat,fsq_bus_id,venue,latitude,longitude,distance,city,source,query,category
0,Italian Restaurants,4d306dd82748b60c62b6dba0,Trattoria Mercatto,43.654453,-79.380974,224,toronto,fsq,NaN,bars
1,Pubs,4ade47b7f964a520997421e3,Duke of Richmond,43.652267,-79.381009,205,toronto,fsq,NaN,bars
2,Bars,4c747ea4604a370462548049,Trios Bistro,43.654910,-79.382987,188,toronto,fsq,NaN,bars
3,Hotel Bars,4cdc78defc9737042ba2d505,Club Lounge,43.651257,-79.383926,227,toronto,fsq,NaN,bars
4,Bars,4f23983ee4b0476578d9c52f,The Supermarket,43.653247,-79.381865,106,toronto,fsq,NaN,bars


In [4]:
create_fsq_table = """
CREATE TABLE IF NOT EXISTS users (
  id INTEGER PRIMARY KEY AUTOINCREMENT,
  venue TEXT NOT NULL,
  latitude REAL,
  longitude REAL,
  distance INTEGER,
  city TEXT NOT NULL,
  nationality TEXT
);
"""

In [5]:
execute_query(connection, create_users_table)  

Query executed successfully


In [9]:
create_posts_table = """
CREATE TABLE IF NOT EXISTS posts(
  id INTEGER PRIMARY KEY AUTOINCREMENT, 
  title TEXT NOT NULL, 
  description TEXT NOT NULL, 
  user_id INTEGER NOT NULL, 
  FOREIGN KEY (user_id) REFERENCES users (id)
);
"""

In [10]:
execute_query(connection, create_posts_table)

Query executed successfully


In [6]:
create_comments_table = """
CREATE TABLE IF NOT EXISTS comments (
  id INTEGER PRIMARY KEY AUTOINCREMENT, 
  text TEXT NOT NULL, 
  user_id INTEGER NOT NULL, 
  post_id INTEGER NOT NULL, 
  FOREIGN KEY (user_id) REFERENCES users (id) FOREIGN KEY (post_id) REFERENCES posts (id)
);
"""

create_likes_table = """
CREATE TABLE IF NOT EXISTS likes (
  id INTEGER PRIMARY KEY AUTOINCREMENT, 
  user_id INTEGER NOT NULL, 
  post_id integer NOT NULL, 
  FOREIGN KEY (user_id) REFERENCES users (id) FOREIGN KEY (post_id) REFERENCES posts (id)
);
"""

execute_query(connection, create_comments_table)  
execute_query(connection, create_likes_table) 

Query executed successfully
Query executed successfully


In [7]:
create_users = """
INSERT INTO
  users (name, age, gender, nationality)
VALUES
  ('James', 25, 'male', 'USA'),
  ('Leila', 32, 'female', 'France'),
  ('Brigitte', 35, 'female', 'England'),
  ('Mike', 40, 'male', 'Denmark'),
  ('Elizabeth', 21, 'female', 'Canada');
"""

execute_query(connection, create_users)   

Query executed successfully


In [11]:
create_posts = """
INSERT INTO
  posts (title, description, user_id)
VALUES
  ("Happy", "I am feeling very happy today", 1),
  ("Hot Weather", "The weather is very hot today", 2),
  ("Help", "I need some help with my work", 2),
  ("Great News", "I am getting married", 1),
  ("Interesting Game", "It was a fantastic game of tennis", 5),
  ("Party", "Anyone up for a late-night party today?", 3);
"""

execute_query(connection, create_posts)  

Query executed successfully


In [12]:
create_posts = """
INSERT INTO
  posts (title, description, user_id)
VALUES
  ("Happy", "I am feeling very happy today", 1),
  ("Hot Weather", "The weather is very hot today", 2),
  ("Help", "I need some help with my work", 2),
  ("Great News", "I am getting married", 1),
  ("Interesting Game", "It was a fantastic game of tennis", 5),
  ("Party", "Anyone up for a late-night party today?", 3);
"""

execute_query(connection, create_posts)  

Query executed successfully


In [13]:
create_comments = """
INSERT INTO
  comments (text, user_id, post_id)
VALUES
  ('Count me in', 1, 6),
  ('What sort of help?', 5, 3),
  ('Congrats buddy', 2, 4),
  ('I was rooting for Nadal though', 4, 5),
  ('Help with your thesis?', 2, 3),
  ('Many congratulations', 5, 4);
"""

create_likes = """
INSERT INTO
  likes (user_id, post_id)
VALUES
  (1, 6),
  (2, 3),
  (1, 5),
  (5, 4),
  (2, 4),
  (4, 2),
  (3, 6);
"""

execute_query(connection, create_comments)
execute_query(connection, create_likes) 

Query executed successfully
Query executed successfully


In [14]:
def execute_read_query(connection, query):
    cursor = connection.cursor()
    result = None
    try:
        cursor.execute(query)
        result = cursor.fetchall()
        return result
    except Error as e:
        print(f"The error '{e}' occurred")

In [15]:
select_users = "SELECT * from users"
users = execute_read_query(connection, select_users)

for user in users:
    print(user)

(1, 'James', 25, 'male', 'USA')
(2, 'Leila', 32, 'female', 'France')
(3, 'Brigitte', 35, 'female', 'England')
(4, 'Mike', 40, 'male', 'Denmark')
(5, 'Elizabeth', 21, 'female', 'Canada')


In [16]:
update_post_description = """
UPDATE
  posts
SET
  description = "The weather has become pleasant now"
WHERE
  id = 2
"""

execute_query(connection, update_post_description)

Query executed successfully


In [17]:
delete_comment = "DELETE FROM comments WHERE id = 5"
execute_query(connection, delete_comment)

Query executed successfully


# MISC

In [13]:
# test Foursquare credentials
test_url = 'https://api.foursquare.com/v2/venues/search'
test_fsq = re.get(test_url, 
                  params={'client_id':fsq_id, 
                          'client_secret':fsq_secret,
                          'll':'40.7,-74',
                          'v':'20210302'})

In [321]:
test_fsq.status_code

NameError: name 'test_fsq' is not defined

#### Build SQLite DB

In [7]:
def Create_Pandas(data_json):
    data = pd.read_json(json.dumps(data_json[list(data_json.keys())[1]])).T
    data.index = pd.to_datetime(data.index)
    
    return data

def Get_TimeSeries(stock, period):
    """
    A wrapper function to obtain the the time series of a the desired stock from the Alpha Vantage API.
    
    Parameters:
    -----------
    stock: the desired stock symbol (e.g., "GOOG", "AMD", "FB", "INTC").
    period: "daily", "weekly", "monthly"
    
    Returns:
    --------
    A pandas dataframe with the columns `open`, `high`, `low`, `close`, and `volume`.
    """
    
    # Let's process the period parameter 
    # which affects the function parameter in the API
    # (I'm using a dictionary here so if the user try a period different from
    # daily, weekly or monthly, an exception will be raised)
    ts_string = "TIME_SERIES_"
    _function = {'DAILY': ts_string + "DAILY",
                 'WEEKLY': ts_string + "WEEKLY",
                 'MONTHLY': ts_string + "MONTHLY"}
    
    # Now let's create a dictionary to hold the parameters of 
    # our request.
    params = {"function": _function[period.upper()],
              "symbol": stock.upper(),
              "outputsize": "full"}
    
    # Requesting the data
    response = data_request(params)
    
    # Check the result of the request
    if response.status_code != 200:
        raise Exception(response.reason)
    if "Error Message" in response.json():
        raise ValueError(response.json()["Error Message"])
        
    # Create the pandas dataframe
    data_json = response.json()
    data = Create_Pandas(data_json)
    data.Stock = data_json['Meta Data']['2. Symbol']
    data.LastUpdate = pd.Period(data_json['Meta Data']['3. Last Refreshed'], freq='D')
    data.columns = [re.split("[1-9]*\. ", column)[1] for column in data.columns] 
    
    return data

In [313]:
def get_data(source, city, query=None, category=None):
    
    c = {'vancouver':('49.282730','-123.120735'),
    'toronto':('43.653225','-79.383186'),
    'nyc':('40.712776','-74.005974'),
    'new westminster':('49.205719','-122.910957')}
    
    try:     
        latitude, longitude = c[city.lower()]
        if source == 'yelp':
            
            p={'latitude':latitude,
               'longitude':longitude,
               'radius':'250',
               'limit':'5'} #remove for ACTUAL run
            
            if (category != None):
                p['categories'] = ol_categories[ol_categories['overlap']==category]['yelp_name'].iat[0]
            if (query != None):
                p['term'] = query
            
            data = re.get('https://api.yelp.com/v3/businesses/search',
                         headers={'Authorization':f'Bearer {yelp_key}'},
                         params=p)
            
            data = data.json()
            
        elif source == 'fsq':
            
            p = {'ll':f'{latitude},{longitude}',
                'radius':'250',
                'limit':'5'} #remove for ACTUAL run}
            
            if (category != None):
                p['category'] = ol_categories[ol_categories['overlap']==category]['fsq_id'].iat[0]
            if (query != None):
                p['query'] = ol_categories[ol_categories['overlap']==query]['fsq_name'].iat[0]
            
            data = client.venues.search(params=p)

        else:
            print('try different source (yelp or fsq)')
    except:
        print('some error occurred')
    
    return data, city.lower(), source.lower(), query.lower()